In [ ]:
from aust_covid.model import build_model
from aust_covid.calibration import get_priors, get_targets
from emutools.inputs import load_param_info
from aust_covid.inputs import get_ifrs
from autumn.infrastructure.remote import springboard
import arviz as az
from emutools.tex import DummyTexDoc, StandardTexDoc
from inputs.constants import INPUTS_PATH, SUPPLEMENT_PATH
import numpy as np
from aust_covid.plotting import plot_dispersion_examples
from aust_covid.utils import add_image_to_doc

In [ ]:
dummy_doc = DummyTexDoc()
epi_model = build_model(dummy_doc)

In [ ]:
rts = springboard.task.RemoteTaskStore()
rts.cd('projects/aust_covid/base_case_analysis')
mt = rts.get_managed_task('2023-08-30T1608-try_DEMZ_10k')
mt.download_all()
idata = az.from_netcdf(mt.local / 'output/calibration_out.nc')

In [ ]:
prior_names = [p.name for p in get_priors()]
targets = get_targets(dummy_doc)
param_info = load_param_info(INPUTS_PATH / 'parameters.yml')
param_info['value'].update(get_ifrs(dummy_doc))
parameters = param_info['value'].to_dict()

In [ ]:
colours = {'notifications_ma': '10, 10, 100', 'deaths_ma': '100, 10, 10'}
centiles = np.linspace(0.1, 0.9, 9)
fig = plot_dispersion_examples(idata, epi_model, parameters, prior_names, targets, colours, centiles)
caption = 'Examples of the effect of values of the negative binomial distribution dispersion parameter.'
print(caption)
fig

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'dispersion', 'Target dispersion', 'austcovid')
add_image_to_doc(fig, 'dispersion_examples', caption, app_doc, 'Calibration')